## Combining CSVs: Useful Code Attempt 1

### Goals for this script:

* Make a chunk of code that does something useful

* Be written in such a way so as to be universal, variables rather than exact names so there are minimal changes required by user

* Count number of controls as an additional bonus at the end

* Determine if csvs are of the same length, and alphebetize

* Check for symmetry based on sample name and remove any mismatches, then output a list of what those are is a separate file

* Combine two CSVs based on sample name and output a separate merged file

In [15]:
#Importing packages:

import Bio as BP
import pandas as pd
import numpy as np
import collections as col
import os
import csv
import re
import regex

In [11]:
cd /Users/anthea.mitchell/Desktop/Coding/CSVs/the_great_merge

/Users/anthea.mitchell/Desktop/Coding/CSVs/the_great_merge


In [17]:
'''Makes all sample names uppercase and alphebetized'''

def upper_alphebetize1(CSV_1, CSV_2):
    CSV_1['Name'] =CSV_1['Name'].str.upper()
    CSV_1 = CSV_1.sort_values('Name')
    CSV_2['Name'] =CSV_2['Name'].str.upper()
    CSV_2 = CSV_2.sort_values('Name')
    return CSV_1, CSV_2


In [18]:
'''Remove nan values that otherwise cause errors in combining along rows and replace with ? value instead.'''
def make_nan_question(Ordered_CSV_1, Ordered_CSV_2):
    Ordered_nan_CSV_1=Ordered_CSV_1.fillna('?')
    Ordered_nan_CSV_2=Ordered_CSV_2.fillna('?')
    return(Ordered_nan_CSV_1, Ordered_nan_CSV_2)

In [19]:
'''Notifies user if sample names are the same by adding to a list and comparing the lists'''

def compare_lists(CSV_1, CSV_2):
    List1 = CSV_1['Name'].tolist()
    List2 = CSV_2['Name'].tolist()
    if List1 != List2:
        print("\nOh no! Your sample names don't match. This could mean there are extra samples on one or both lists, or simply that the names of samples are different. Proceed with caution and look closely at any changes made by script.")
        confirm = input("\nType 'OK' to acknowledge. ")
    if List1 == List2:
        print("\nGo ahead! Everything matches!")
    return List1, List2

In [20]:
'''Add items under column header "Name" as items in a list, creating a list for each CSV'''

def make_lists(CSV_1, CSV_2):
    List1 = CSV_1['Name'].tolist()
    List2 = CSV_2['Name'].tolist()
    return List1, List2

In [21]:
'''This code won't work if there are duplicates on the list, and while I played with automating the removal
of duplicates, in the end it's usually a case where some human intervention is needed. Why is there a duplicate?
Did someone scan something twice by accident? Is it literally just a case of copy pasting twice? Are they genuinely
the same sample from a longitudinal study that simply need dates appended?'''

def duplicate_search(List1, List2):
    x = 0
    Compare = col.Counter(List1)
    for i in Compare:
        x = x + Compare[i]
        if Compare[i]>1:
            print("\nIn CSV_1 There is a repeated sample name called", i, "and you need to change or remove it before you continue.")
        elif x == len(List1):
            print("\nCSV_1 has no duplicates!")
    x = 0
    Compare = col.Counter(List2)
    for i in Compare:
        x = x + Compare[i]
        if Compare[i]>1:
            print("\nIn CSV_2 there is a repeated sample name called", i, "and you need to change or remove it before you continue.")
        elif x == len(List1):
            print("\nCSV_2 has no duplicates!")
    

In [22]:
'''Learning experience: Nested for loops were a reminder of the importance of scope,a review of boolians, 
and good practice at indenting correctly. It compares each index in list 1 to each index in 
list 2, and then only after looping all the way through list 2 does it add an index to the set. 
Sets were also a new concept and are great because they do not accept duplicates.'''

def loop_through_list_1(List1, List2):
    new_set = set()
    bad_set_1 = set()
    for a in range(len(List1)):
        flag = False
        for b in range(len(List2)):
            if List1[a] == List2[b]:
                flag = True

        if flag == True:
            new_set.add(List1[a])
            
        if flag == False:
            bad_set_1.add(List1[a])
            
    return bad_set_1, new_set

In [23]:
'''This does the same, except in the other direction, this time comparing CSV2 to CSV1.'''
def loop_through_list_2(List2, List1, bad_set_1, new_set):
    for a in range(len(List2)):
        flag = False
        for b in range(len(List1)):
            if List2[a] == List1[b]:
                flag = True

        if flag == True:
            new_set.add(List2[a])
            
        if flag == False:
            bad_set_1.add(List2[a])
            
    return bad_set_1, new_set

In [24]:
'''Outputs the list of samples that don't appear on BOTH csvs as a file'''

def make_bad_list_CSV(bad_list):
    BL = pd.DataFrame(bad_list)
    #removed_samples = bad_list.pop(0)
    #df = pd.DataFrame(bad_list, columns = removed_samples)
    BL.to_csv(str(input("\nWhat do you want to call your output list of removed samples? (don't forget to write '.csv' at the end.) ")))

In [25]:
'''Creates a fresh csv with all the rows merged by Name, selecting only those 
that appeared on both of the original CSVs.'''

def create_great_merge_CSV_1(Ordered_nan_CSV_1, Ordered_nan_CSV_2, good_list):

    new_df_1 = Ordered_nan_CSV_1[Ordered_nan_CSV_1.Name.isin(good_list)]
    new_df_2 = Ordered_nan_CSV_2[Ordered_nan_CSV_2.Name.isin(good_list)]
    
    MergeFile = pd.merge(new_df_1, new_df_2, left_on =["Name"], right_on = ["Name"])
    
    return MergeFile
    


In [26]:
'''Early csvs were run through code that replaced nan values with ? value, and this
now removes those ? values so that the final csv has blank spaces where appropriate.'''
def make_question_nan_again(merge_with_question):
    merge_with_question = merge_with_question.replace('?', np.NaN)
    
    return merge_with_question.to_csv(input("\nWhat do you want to call you new merged file? (don't forget .csv at the end) "))

In [27]:
def main():
    print("This script takes two CSVs and merges them based on column heading. Please format your CSV so that names of samples are under a column called 'Name'.")
    
    try:
        CSV_1 = pd.read_csv(input("\nWhat is your first csv file? (CSV_1) "))
    except:
        print("\nYou must have mistyped your file name, try again.")
        CSV_1 = pd.read_csv(input("\nWhat is your first csv file? (CSV_1) "))
    try:
        CSV_2 = pd.read_csv(input("\nWhat is your second csv file? (CSV_2) "))
    except:
        print("\nYou must have mistyped your file name, try again.")
        CSV_2 = pd.read_csv(input("\nWhat is your second csv file? (CSV_2) "))

    Ordered_CSV_1, Ordered_CSV_2 = upper_alphebetize1(CSV_1, CSV_2)
    Ordered_nan_CSV_1, Ordered_nan_CSV_2 = make_nan_question(Ordered_CSV_1, Ordered_CSV_2)
    List1, List2 = make_lists(Ordered_nan_CSV_1, Ordered_nan_CSV_2)
    duplicate_search(List1, List2)
    compare_lists(CSV_1, CSV_2)
    bad_set_1, new_set = loop_through_list_1(List1, List2)
    bad_set, good_set =loop_through_list_2(List2, List1, bad_set_1, new_set)
    
    bad_list = list(bad_set)
    good_list = list(good_set)
    
    make_bad_list_CSV(bad_list)
    merge_with_question = create_great_merge_CSV_1(Ordered_nan_CSV_1, Ordered_nan_CSV_2, good_list)
    make_question_nan_again(merge_with_question)
    
    print ("All done! Check your folder for new files!")
    

In [30]:
main()

This script takes two CSVs and merges them based on column heading. Please format your CSV so that names of samples are under a column called 'Name'.

What is your first csv file? (CSV_1) positive_cases.csv

What is your second csv file? (CSV_2) Sample_CTS.csv

In CSV_1 There is a repeated sample name called BX23WKEA9 and you need to change or remove it before you continue.

In CSV_1 There is a repeated sample name called BX4F52M57 and you need to change or remove it before you continue.

In CSV_1 There is a repeated sample name called BX7KL1JAU and you need to change or remove it before you continue.

In CSV_1 There is a repeated sample name called BXK54P61Q and you need to change or remove it before you continue.

In CSV_1 There is a repeated sample name called BXSNL7K9D and you need to change or remove it before you continue.

In CSV_1 There is a repeated sample name called BXWUNRRN7 and you need to change or remove it before you continue.

CSV_1 has no duplicates!

In CSV_2 there i

### Listing and Counting Controls or Special Samples:

In [ ]:
'''Should you need to search for certain control type names, this can be modified to search
and output how many are present.'''

CSV_1_String = str(CSV_1)
NTC = re.findall("NTC", CSV_1_String)
print(NTC)
print(len(NTC))
